In [ ]:

!pip install -q langchain-openai

In [ ]:
API_KEY="9a6a43f55cef45248997ac28aaf5f3da"

In [ ]:

from langchain_openai import AzureChatOpenAI
model = AzureChatOpenAI(
        azure_endpoint="https://eygroup-0003-new.openai.azure.com/",
        azure_deployment="gpt-35-turbo-16k",
        openai_api_version="2024-06-01",
        api_key=API_KEY
    )

In [ ]:

res = model.invoke("Hi! Tell me a joke about noodles.")

In [ ]:

res.content

"Sure, here's a noodle joke for you:\n\nWhy did the noodle go to the gym?\n\nBecause it wanted to get shredded!"

In [ ]:

!pip install langchain-community langchain-core

In [ ]:
!pip install -qU langchain_community pypdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

In [ ]:

!ls

'Mantis shrimp..pdf'   sample_data


In [ ]:
pdf_file = "/content/sample_data/RAG/LiteraryMovements.pdf"
loader = PyPDFLoader(pdf_file)

docs = loader.load()

In [ ]:

print("len:",len(docs),"\n","content:",docs[0].page_content)

len: 2 
 content: Literary Movements  
Literary movements are styles or trends in literature that share common characteristics 
and themes, often reflecting the social and political climates of their times. Here are a few 
significant literary movements:  
1. Classicism  (5th century BC - 18th century): Emphasized clarity, order, balance, and 
harmony. Notable works include those by Homer, Virgil, and during the 
Enlightenment, writers like Voltaire.  
2. Romanticism  (late 18th century - mid-19th century): Focused on emotion, nature, 
individualism, and the sublime. Key figures include William Wordsworth, Mary 
Shelley, and Edgar Allan Poe.  
3. Realism  (mid-19th century - early 20th century): Sought to depict everyday life and 
society without romanticization. Authors like Gustave Flaubert, Leo Tolstoy, and 
Mark Twain were prominent.  
4. Modernism  (late 19th century - mid-20th century): Characterized by a break with 
traditional forms and techniques, exploring fragmented narrativ

In [ ]:
!pip install -qU langchain-openai

In [ ]:
AZURE_OPENAI_ENDPOINT = "https://eygroup-0003-new.openai.azure.com/"
AZURE_OPENAI_EMBEDDINGS_ENDPOINT="https://eygroup-0003-new.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15"
AZURE_OPENAI_DEPLOYMENT_NAME = "gpt-35-turbo-16k"
AZURE_OPENAI_API_VERSION = "2024-06-01"
AZURE_OPENAI_API_KEY = API_KEY

In [ ]:
import os
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = AZURE_OPENAI_DEPLOYMENT_NAME
os.environ["AZURE_OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION
os.environ["AZURE_OPENAI_EMBEDDINGS_ENDPOINT"] = AZURE_OPENAI_EMBEDDINGS_ENDPOINT
os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY


In [ ]:

from langchain_openai import AzureChatOpenAI

In [ ]:
model = AzureChatOpenAI(
    azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version = os.environ["AZURE_OPENAI_API_VERSION"]
)

In [ ]:
!pip install -qU langchain_chroma

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import AzureOpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
print(splits)
embeddings = AzureOpenAIEmbeddings(azure_endpoint=os.environ["AZURE_OPENAI_EMBEDDINGS_ENDPOINT"])

vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

retriever = vectorstore.as_retriever()

[Document(metadata={'source': '/content/sample_data/RAG/LiteraryMovements.pdf', 'page': 0}, page_content='Literary Movements  \nLiterary movements are styles or trends in literature that share common characteristics \nand themes, often reflecting the social and political climates of their times. Here are a few \nsignificant literary movements:  \n1. Classicism  (5th century BC - 18th century): Emphasized clarity, order, balance, and \nharmony. Notable works include those by Homer, Virgil, and during the \nEnlightenment, writers like Voltaire.  \n2. Romanticism  (late 18th century - mid-19th century): Focused on emotion, nature, \nindividualism, and the sublime. Key figures include William Wordsworth, Mary \nShelley, and Edgar Allan Poe.  \n3. Realism  (mid-19th century - early 20th century): Sought to depict everyday life and \nsociety without romanticization. Authors like Gustave Flaubert, Leo Tolstoy, and \nMark Twain were prominent.  \n4. Modernism  (late 19th century - mid-20th cen

In [ ]:
print("len:",len(docs),"\n","content:",docs[0].page_content)

len: 2 
 content: Literary Movements  
Literary movements are styles or trends in literature that share common characteristics 
and themes, often reflecting the social and political climates of their times. Here are a few 
significant literary movements:  
1. Classicism  (5th century BC - 18th century): Emphasized clarity, order, balance, and 
harmony. Notable works include those by Homer, Virgil, and during the 
Enlightenment, writers like Voltaire.  
2. Romanticism  (late 18th century - mid-19th century): Focused on emotion, nature, 
individualism, and the sublime. Key figures include William Wordsworth, Mary 
Shelley, and Edgar Allan Poe.  
3. Realism  (mid-19th century - early 20th century): Sought to depict everyday life and 
society without romanticization. Authors like Gustave Flaubert, Leo Tolstoy, and 
Mark Twain were prominent.  
4. Modernism  (late 19th century - mid-20th century): Characterized by a break with 
traditional forms and techniques, exploring fragmented narrativ

In [ ]:

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

In [ ]:
context_system_prompt = (
    "Literature Companion System"
    "people find Book Recommendations, "
    "people find Literary Eras and Movements "
    "people find Author Profiles"
)

In [ ]:
messages = [
    ("system", context_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
]
prompt = ChatPromptTemplate.from_messages(messages)

In [ ]:
context_retriever = create_history_aware_retriever(
    llm=model, retriever=retriever, prompt=prompt
)

In [ ]:
system_prompt = (
    "You are a literature Companion system"
    "You love literature literary movements  "
    "What is your favorite movements ?"
    "\n\n"
   "{context}"
)

In [ ]:

system_prompt

'You are a literature Companion systemYou love literature literary movements  What is your favorite movements ?\n\n{context}'

In [ ]:
conversation_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [ ]:
context_chain = create_stuff_documents_chain(
    model,
    conversation_prompt
)

In [ ]:
rag_chain = create_retrieval_chain(context_retriever,context_chain)

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


In [ ]:
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
conversational_rag_chain.invoke(
    {"input": "What book you recommanad?"},
    config={
        "configurable": {"session_id": "eygrp3"}
    },
)["answer"]

'As an AI, I don\'t have personal preferences, but I can recommend some widely acclaimed books from different genres:\n\n1. Fiction: "To Kill a Mockingbird" by Harper Lee - A powerful novel that addresses themes of racial injustice, morality, and compassion.\n\n2. Science Fiction: "1984" by George Orwell - A dystopian novel that explores the dangers of totalitarianism and the importance of individual freedom.\n\n3. Fantasy: "The Lord of the Rings" by J.R.R. Tolkien - An epic high fantasy series that takes readers on a journey through a richly imagined world.\n\n4. Classic Literature: "Pride and Prejudice" by Jane Austen - A witty and insightful novel that examines societal norms, love, and the complexities of human relationships.\n\n5. Historical Fiction: "The Book Thief" by Markus Zusak - Set during World War II, this novel tells the story of a young girl who finds solace in books and the power of words.\n\n6. Non-Fiction: "Sapiens: A Brief History of Humankind" by Yuval Noah Harari -

In [ ]:

store

{'eygrp3': InMemoryChatMessageHistory(messages=[HumanMessage(content='What book you recommanad?'), AIMessage(content='As an AI, I don\'t have personal preferences, but I can recommend some widely acclaimed books from different genres:\n\n1. Fiction: "To Kill a Mockingbird" by Harper Lee - A powerful novel that addresses themes of racial injustice, morality, and compassion.\n\n2. Science Fiction: "1984" by George Orwell - A dystopian novel that explores the dangers of totalitarianism and the importance of individual freedom.\n\n3. Fantasy: "The Lord of the Rings" by J.R.R. Tolkien - An epic high fantasy series that takes readers on a journey through a richly imagined world.\n\n4. Classic Literature: "Pride and Prejudice" by Jane Austen - A witty and insightful novel that examines societal norms, love, and the complexities of human relationships.\n\n5. Historical Fiction: "The Book Thief" by Markus Zusak - Set during World War II, this novel tells the story of a young girl who finds sola

In [ ]:
while True:
  user_input = input("Enter 'q' to quit or anything else to continue: ")
  if user_input == 'q':
    break
  else:
    res = conversational_rag_chain.invoke(
    {"input": user_input},
    config={
        "configurable": {"session_id": "myuniqueid001"}
    },
    )["answer"]
    print(res)


Enter 'q' to quit or anything else to continue: Can you recommand books
Certainly! Here are some book recommendations from various genres:

1. Fiction:
- "To Kill a Mockingbird" by Harper Lee
- "1984" by George Orwell
- "Pride and Prejudice" by Jane Austen
- "The Great Gatsby" by F. Scott Fitzgerald
- "The Catcher in the Rye" by J.D. Salinger

2. Mystery/Thriller:
- "Gone Girl" by Gillian Flynn
- "The Girl with the Dragon Tattoo" by Stieg Larsson
- "The Da Vinci Code" by Dan Brown
- "The Girl on the Train" by Paula Hawkins
- "And Then There Were None" by Agatha Christie

3. Science Fiction/Fantasy:
- "Dune" by Frank Herbert
- "The Hobbit" by J.R.R. Tolkien
- "Ender's Game" by Orson Scott Card
- "The Hunger Games" by Suzanne Collins
- "A Wrinkle in Time" by Madeleine L'Engle

4. Non-Fiction:
- "Sapiens: A Brief History of Humankind" by Yuval Noah Harari
- "Becoming" by Michelle Obama
- "The Immortal Life of Henrietta Lacks" by Rebecca Skloot
- "Educated" by Tara Westover
- "The Power of